In [ ]:
# !pip install openai

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

weaviate_url = os.getenv("WEAVIATE_URL")
openai_key = os.getenv("OPENAI_API_KEY")
openai_url = os.getenv("OPENAI_URL")
project = os.getenv("AZURE_PROJECT")

print(weaviate_url, openai_key, openai_url, project)

## Generate query from prompt

In [ ]:
from openai import OpenAI

openai_client = OpenAI(
    api_key=openai_key,
    # TODO: make sure this works with a custom openai url
    base_url=f"{openai_url}/v1",
    project=project
)

def generate_query_from_promt(prompt):
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            { "role": "system", "content": "Your job is to extract a query from the provided user prompt, the query will then be used to run a query in a vector database." },
            { 
                "role": "user",
                "content": f"Please give me a 2-3 word query that can be used to find relevant info to the following prompt - {prompt}" },
        ]
    )
    return response.choices[0].message.content

In [ ]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

## Connect to Weaviate

In [ ]:
import weaviate
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_local(
    host=weaviate_url,

    headers={
        "X-OpenAI-Api-Key": openai_key,
    },

    # connect with timeout config
    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=30, query=60, insert=120)  # Values in seconds
    # )
)

client.is_ready()

## Two-step RAG

In [ ]:
def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get("Wiki")

    response = wiki.generate.near_text(
        query=generated_query,
        limit=3,
        grouped_task=prompt,
        grouped_properties=["text", "title"]
    )

    # Print results
    print("\n=== Generated Response ===")
    print(response.generated)

    print("\n=== Source ===")
    for item in response.objects:
        print(item.properties)

In [ ]:
# two_step_rag("What wild animals do we know about?")
two_step_rag("Please provide an explanation at a highschool level. How do airplanes fly?")

In [ ]:
two_step_rag("What are the pros and cons of automation using computer?")

In [ ]:
two_step_rag("How do CPUs work?")

## Close the client

In [ ]:
client.close()